<a href="https://colab.research.google.com/github/tobbyyan-barker/machinelearning111/blob/main/KNN%20and%20NN%20using%20CIFAR10%20datasets%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz


--2025-07-20 01:49:12--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  99.5MB/s    in 1.6s    

2025-07-20 01:49:14 (99.5 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [ ]:
!tar -zxvf cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [ ]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

# 1. 定义我们要读取的文件的“地址”（路径）
# 因为 data_batch_1 文件在 cifar-10-batches-py 文件夹里，所以路径是这样的
file_path = 'cifar-10-batches-py/data_batch_1'

# 2. 调用函数，把地址告诉它，它会返回一个字典
batch_1_data = unpickle(file_path)

# 3. 检查一下我们是否成功拿到了数据
print("成功读取！数据类型是:", type(batch_1_data))
print("字典里包含的键:", batch_1_data.keys())

images = batch_1_data[b'data']
print(images)
labels = batch_1_data[b'labels']
print(labels)
print("第一批数据的图片数量:", len(labels))

成功读取！数据类型是: <class 'dict'>
字典里包含的键: dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
[[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]
 [255 253 253 ...  83  83  84]
 ...
 [ 71  60  74 ...  68  69  68]
 [250 254 211 ... 215 255 254]
 [ 62  61  60 ... 130 130 131]]
[6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1

# 新段落

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#近邻算法代码---核心: 就是找到和当前图像距离最小的点 训练死记硬背 预测勤能补拙

#导入相应的库:
import numpy as np
from tensorflow.keras.datasets import cifar10

def load_CIFAR10_from_keras():
    """
    使用 Keras 库加载 CIFAR-10 数据集。
    Keras 会自动处理下载和解压。
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # Keras 加载的 y_train 和 y_test 是二维数组 (50000, 1)，需要转换成一维数组
    return x_train, y_train.flatten(), x_test, y_test.flatten()
#加载训练图片，训练标签，预测图片，预测标签
Xtr,Ytr,Xtl,Ytl = load_CIFAR10_from_keras()

#把图像数据拉伸成行向量: 电脑没办法直接看懂彩色图片
Xtr_rows = Xtr.reshape(Xtr.shape[0],32*32*3)
Xtl_rows = Xtl.reshape(Xtl.shape[0],32*32*3)

#自定义分类器: 分类器包含构造函数， 训练函数 和 预测函数 --- 有这么多函数:写自定义类
class Nearest_Neighbourhood(object):
  def __init__(self):
    pass

  def training(self,x,y):
    self.Xtr_rows = x
    self.Ytr = y

  def predicting(self,X):
    num_length = X.shape[0]
    Y_pred = np.zeros(num_length,dtype = self.Ytr.dtype)
    for i in range(num_length):
      distance = np.sum(abs(self.Xtr_rows - X[i,:]), axis = 1)
      min_index = np.argmin(distance)
      Y_pred[i] = self.Ytr[min_index]

    return Y_pred

#实例化nn对象，调用类里面的训练和预测函数来完成:
nn=Nearest_Neighbourhood()
nn.training(Xtr_rows,Ytr)
Y_predict = nn.predicting(Xtl_rows)
print(f"accurancy is {np.mean(Y_predict==Ytl)}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


KeyboardInterrupt: 

In [ ]:
# 导入相应的库
import numpy as np
import pickle
import os

# ================================================================
# 1. 实现 load_CIFAR10 函数
# ================================================================
def unpickle(file):
    """一个辅助函数，用来读取pickle格式的文件"""
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

def load_CIFAR10(ROOT):
    """
    加载并整合CIFAR-10数据集
    ROOT: 存放数据集文件夹cifar-10-batches-py的根目录
    """
    xs = []
    ys = []
    # 循环加载5个训练批次文件
    for b in range(1, 6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b,))
        X = unpickle(f)
        xs.append(X[b'data'])
        ys.append(X[b'labels'])

    # 将5个批次的数据整合到一个大的numpy数组中
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del xs, ys

    # 加载测试文件
    Xte = unpickle(os.path.join(ROOT, 'test_batch'))
    Yte = np.array(Xte[b'labels'])
    Xte = Xte[b'data']

    return Xtr, Ytr, Xte, Yte

# ================================================================
# 2. 自定义分类器 (修正了内部错误，并使用了更通用的方法名 train/predict)
# ================================================================
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, x, y):
        """ 训练阶段：直接记住所有数据 """
        self.X_train = x
        self.y_train = y

    def predict(self, X_test,k):
        """ 预测阶段：为每一张测试图片寻找最近邻 """
        num_test = X_test.shape[0]
        y_pred = np.zeros(num_test, dtype=self.y_train.dtype)

        for i in range(num_test):

            distances = np.sum(np.abs(self.X_train - X_test[i,:]), axis=1)

            k_min_indexes = np.argsort(distances)[:k]

            k_nearest_labels = self.y_train[k_min_indexes]

            y_pred[i] = np.argmax(np.bincount(k_nearest_labels))
            # 加上一个打印输出，方便我们看到进度
            if (i + 1) % 100 == 0:
                print(f"Predicted {i+1}/{num_test} images for k = {k}")

        return y_pred

# ================================================================
# 3. 主程序流程
# ================================================================
# **重要**: 请将这里的路径改成你存放 cifar-10-batches-py 文件夹的路径
# 在Colab中，如果解压在主目录，路径就是 'cifar-10-batches-py'
cifar10_dir = 'cifar-10-batches-py'

# 加载数据
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

print(f"原始数据的形状是:{X_train.shape}")

# 打印数据形状，确认加载正确
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

# 把图像数据拉伸成行向量 (这一步其实在load_CIFAR10里已经包含了，数据本身就是3072维的)
# 如果加载的是原始图片，这一步是必须的。但CIFAR-10的数据已经是拉伸好的了。

#数据准备和分割
x_train_subset = X_train[:5000,:]
y_train_subset = y_train[:5000]
x_val_subset = X_train[5000:5500,:]
y_val_subset = y_train[5000:5500]
x_test_subset = X_test[:100,:]
y_test_subset = y_test[:100]

#开始验证吧:
validation_accur = []

k_list = [1,3,5,7,10,100,1000]

mm = NearestNeighbor()

#训练只需要一次
mm.train(x_train_subset,y_train_subset)

#验证需要验证很多次

for k in k_list:
  y_val_prediction = mm.predict(x_val_subset,k)
  acc = np.mean(y_val_prediction == y_val_subset)
  print(f" acc is {acc} and k is {k}")
  validation_accur.append((k,acc))

best_k_tuples = max(validation_accur,key=lambda item: item[1])
best_k = best_k_tuples[0]
best_accurancy = best_k_tuples[1]

#拿最好的k进行预测:
y_test_prediction = mm.predict(x_test_subset,best_k)
predict_accurancy = np.mean(y_test_prediction == y_test_subset)
print(f" test accurancy is {predict_accurancy} under best k: {best_k}")

原始数据的形状是:(50000, 3072)
Training data shape:  (50000, 3072)
Training labels shape:  (50000,)
Test data shape:  (10000, 3072)
Test labels shape:  (10000,)
Predicted 100/500 images for k = 1
Predicted 200/500 images for k = 1
Predicted 300/500 images for k = 1
Predicted 400/500 images for k = 1
Predicted 500/500 images for k = 1
 acc is 0.21 and k is 1
Predicted 100/500 images for k = 3
Predicted 200/500 images for k = 3
Predicted 300/500 images for k = 3
Predicted 400/500 images for k = 3
Predicted 500/500 images for k = 3
 acc is 0.172 and k is 3
Predicted 100/500 images for k = 5
Predicted 200/500 images for k = 5
Predicted 300/500 images for k = 5
Predicted 400/500 images for k = 5
Predicted 500/500 images for k = 5
 acc is 0.196 and k is 5
Predicted 100/500 images for k = 7
Predicted 200/500 images for k = 7
Predicted 300/500 images for k = 7
Predicted 400/500 images for k = 7
Predicted 500/500 images for k = 7
 acc is 0.192 and k is 7
Predicted 100/500 images for k = 10
Predicted 20